In [ ]:
#矩阵计算其实就是对矩阵求导
# 求导拓展到向量就是梯度
# 当y和x都是标量，那么y对x的求导也是标量
# 当y是向量，x是标量，或者相反，y对x的求导会变为向量
# 如果x和y都是向量，y对x求导会变为矩阵

 1. 当x是列向量，y是标量，导数就是对x向量的每一个值求对应y对xi元素的导数，且结果成为行向量

 举例：y = x1^2 + 2x2^2 y是个标量
        x = [x1,x2]^T    x是个列向量

    求 y对x的导数就是 [2x1,4x2] 的行向量，行向量第一个参数是这么算的：(x1^2 + 2x2^2)对x1求导

    从几何角度理解，就是 x1,x2,y 作为坐标轴的3维空间内存在一个曲面，曲面函数是y = x1^2 + 2x2^2，x是在x1,x2平面的一个向量，这个向量的垂线就是等高线切线。导数就是这个向量方向的增长趋势的向量。

    梯度指向了值变化最大的那个方向

2. 若x是标量，y是列向量，y对x的导数就是个列向量

2. 如果x和y都是列向量
    
举例：x = [x1,x2,x3]^T  
      y= [y1,y2,y3]^T


# 自动求导

自动计算一个函数在指定值上的导数

计算图，一个计算可以拆分多个步骤，成为一个无环图

In [ ]:
# 显示构造

from mxnet import sym

a = sym.var()
b = sym.var()
c = 2*a+b

In [ ]:
# 隐式构造

from mxnet import autograd,nd

with autograd.record():
    a = nd.ones((2,1))
    b = nd.ones((2,1))
    c = 2*a + b

In [1]:
# 计算y关于x的梯度（带方向的导数，结果是个带方向的向量）

import torch
x = torch.arange(4.0) # 创建x向量
# 告诉Tensor存储梯度数据
x.requires_grad_(True) # 等价 `x = torch.arange(4,requires_gard=True)`
print(x)
y = 2*torch.dot(x,x) # x的内积*2，是个标量，结果是28
y

tensor([0., 1., 2., 3.], requires_grad=True)


tensor(28., grad_fn=<MulBackward0>)

In [2]:
# 调用反向传播函数，求出y关于x的每一个分量的导数（梯度）
y.backward()
x.grad # 因为x设置保存了梯度，可以看到y对x的每个分量导数

tensor([ 0.,  4.,  8., 12.])

In [3]:
# 默认情况下，pytorch会累积梯度，所以需要清除以前的梯度
x.grad.zero_()
y = x.sum()
y.backward()
x.grad  # y对x的梯度存放在x.grad是非常合理的，因为是对x的导数，表明x怎么变化y的结果变化率最大，所以放在x.grad很合理


tensor([1., 1., 1., 1.])

In [8]:
# 以下是一个反向传播和一次正向的全部过程，先正向计算所有值，再反向传播
from torchviz import make_dot # 可视化计算图

# 0. 设定w1矩阵，h= w1*x向量 y = h*w2
w1 = torch.tensor([[0.5,1.5],[2.3,3]],dtype=torch.float32,requires_grad=True)
x = torch.tensor([1,0.5],dtype=torch.float32,requires_grad=True)
w2 = torch.tensor([[1.0],[1.0]],requires_grad=True)
t = torch.tensor(4) # 真实值是4
print(x,w1,w2) # 初始的权重都是随机

# 1. 开始前向传播
h = torch.mv(w1,x.T) # w1矩阵和x向量得到h向量，一个二维张量
h = h.reshape((1,2)) # h转为矩阵和w2矩阵相乘
y = torch.mm(h,w2)
print('H:'+str(h))
print('预测：'+str(y.item()))


E = ((t-y)**2)/2# 定义损失函数
print('损失函数结果：',str(E))


# 2. 反向传播并且更新权重

E.backward() # 对损失函数反向传播，得到损失函数对所有参与计算的元素的梯度，即求导

print('w1.grad:'+str(w1.grad)) # 对 w矩阵的梯度，就是对当时的x作为常量带入，求出损失函数对w的每个分量的导数
print('w2.grad:'+str(w2.grad))
# w1有了梯度后，为了使得损失函数更小需要往梯度反方向走，0.01是学习率
w1 = w1 - 0.1*w1.grad
w2 = w2 - 0.1*w2.grad
print(w1)
print(w2)
dot = make_dot(E,params={'x':x,'w1':w1,'w2':w2}) # z关于x的绘制
dot.view()
# 现在我们有了新的权重 w1和w2了，可以再次执行前向


tensor([1.0000, 0.5000], requires_grad=True) tensor([[0.5000, 1.5000],
        [2.3000, 3.0000]], requires_grad=True) tensor([[1.],
        [1.]], requires_grad=True)
H:tensor([[1.2500, 3.8000]], grad_fn=<ReshapeAliasBackward0>)
预测：5.050000190734863
损失函数结果： tensor([[0.5513]], grad_fn=<DivBackward0>)
w1.grad:tensor([[1.0500, 0.5250],
        [1.0500, 0.5250]])
w2.grad:tensor([[1.3125],
        [3.9900]])
tensor([[0.3950, 1.4475],
        [2.1950, 2.9475]], grad_fn=<SubBackward0>)
tensor([[0.8687],
        [0.6010]], grad_fn=<SubBackward0>)


'Digraph.gv.pdf'

In [5]:
x = torch.arange(4.0,requires_grad=True) # 创建x向量
y = x*x # 直接乘法是一种特殊内积，每个元素相乘
# y.backward() 这时候y是个向量，不是个标量，梯度应该是个矩阵，但是在dl中很少对向量的函数求导
y.sum().backward() # 大部分是求和成为了标量再求导
x.grad

tensor([0., 2., 4., 6.])

In [6]:
# 将部分计算移除计算图
from torchviz import make_dot # 可视化计算图
x.grad.zero_()
y = x*x # 这时y是关于x向量的函数
u = y.detach() # 表示u是和x无关的，detach标识脱离，以后u看作常数
z = u * x # 所以这里z对x的导数是看作常数的u，u不会再对x求导
z.sum().backward()
print(x.grad)

dot = make_dot(z,params={'x':x}) # z关于x的绘制
dot.view()  

tensor([0., 1., 4., 9.])


'Digraph.gv.pdf'